# General Walkthrough and Usage of Iter-Active FBA 

Pip installation:<br>
(from PyPI)<br>
PyPI is not yet registered

<br>
(or from the github repo)<br>
pip install "git+https://github.com/m-3-lab/iiFBA.git@main#subdirectory=package"


In [4]:
# load package
import iifba


## Model Loading
This walkthrough will utilize example models installed with the Iter-Active FBA Package. These models follow and are accessible at [VMH AGORA2](https://vmh.life/files/reconstructions/AGORA2/version2.01/mat_files/individual_reconstructions/):
- Bacteroides_thetaiotaomicron_3731.mat
- Escherichia_coli_str_K_12_substr_MG1655.mat



In [ ]:

model, media = load_example_models()


/home/rseag/anaconda3/envs/MBE2/lib/python3.10/site-packages/iifba/AGORA2_Models/Escherichia_coli_str_K_12_substr_MG1655.mat


No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, p
